<a href="https://colab.research.google.com/github/wbigger/2025-vian-sentiment-analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create dataset

In [5]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [6]:
from datasets import load_dataset

In [8]:
v01_files = ["v01/m00 - m00-1.csv",
    "v01/m00 - m00-2.csv",
    "v01/m00 - m00-3.csv",
    "v01/m00 - m00-4.csv",
    "v01/m01 - m01-1.csv",
    "v01/m02 - m02-1.csv",
    "v01/m03 - m03-1.csv",
    "v01/m04 - m04-1.csv"]
dataset = load_dataset("csv",data_files=v01_files)

Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
sample = dataset["train"].shuffle(seed=42).select(range(10))
print(sample[:3])

{'text': ['il 10 che ho preso ad educazione fisica', 'Ho fatto una passeggiata tra le vie del mio paese', 'Professori poco empatici e disattenti alle esigenze dei ragazzi'], 'sentiment': ['positive', 'neutral', 'negative']}


In [10]:
dataset_clean = dataset["train"].train_test_split(train_size=0.8, seed=42)

In [11]:
dataset_clean.save_to_disk('v01/dataset_clean')

Saving the dataset (0/1 shards):   0%|          | 0/279 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/70 [00:00<?, ? examples/s]

# Upload dataset

In [16]:
from huggingface_hub import notebook_login
notebook_login()

In [17]:
dataset_clean.push_to_hub("sentiment-analysis-test")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/wbigger/sentiment-analysis-test/commit/d5120f565f35a278c40268f2102b7c8a2c03849c', commit_message='Upload dataset', commit_description='', oid='d5120f565f35a278c40268f2102b7c8a2c03849c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/wbigger/sentiment-analysis-test', endpoint='https://huggingface.co', repo_type='dataset', repo_id='wbigger/sentiment-analysis-test'), pr_revision=None, pr_num=None)